In [102]:
from htmd import *
import numpy as np
from copy import deepcopy
import sys
import getopt

def read_crd(fname):
    from htmd.molecule.readers import Topology
    t = Topology()

    mol = Molecule()
    coords = list()
    nAtoms = 0
    current_resnum = 1
    up = []
    down = []

    with open(fname, 'r') as f:
        for line in f:
            pieces = line.split()
            if (not pieces[0].startswith("*") and len(pieces) > 7):
                if pieces[3] == 'P':
                    if float(pieces[6]) > 0:
                        up.append(pieces[1])
                    else:
                        down.append(pieces[1])
                t.resid.append(pieces[1])
                t.resname.append(pieces[2])
                t.name.append(pieces[3])
                coords.append([[float(pieces[4])],[float(pieces[5])],[float(pieces[6])]])
                t.segid.append(pieces[7])
                nAtoms = nAtoms + 1

    mol._parseTopology(t, fname)
    mol.coords = np.array(coords, dtype=np.float32)
    mol.filter('not water')
    return(mol, up, down)

In [103]:
my_membrane1 = 'DYPA.crd'
my_membrane2 = 'DGPA.crd'

m1 = read_crd(my_membrane1)
m2 = read_crd(my_membrane2)

residues1 = 'resid ' + ' '.join(m1[2])
residues2 = 'resid ' + ' '.join(m2[1]) 

m1[0].filter(residues1)
m2[0].filter(residues2)


2017-05-30 18:10:08,259 - htmd.molecule.vmdparser - WARNING - More than 3 characters were used for segids. Due to limitations in VMD atomselect segids will be ignored for the atomselection.
2017-05-30 18:10:08,298 - htmd.molecule.molecule - INFO - Removed 3669 atoms. 3024 atoms remaining in the molecule.
2017-05-30 18:10:08,503 - htmd.molecule.vmdparser - WARNING - More than 3 characters were used for segids. Due to limitations in VMD atomselect segids will be ignored for the atomselection.
2017-05-30 18:10:08,543 - htmd.molecule.molecule - INFO - Removed 3072 atoms. 3696 atoms remaining in the molecule.
2017-05-30 18:10:08,590 - htmd.molecule.molecule - INFO - Removed 1512 atoms. 1512 atoms remaining in the molecule.
2017-05-30 18:10:08,645 - htmd.molecule.molecule - INFO - Removed 1848 atoms. 1848 atoms remaining in the molecule.


In [104]:
n = 100
# t = template surface
# us = user surface
surface_m1 = 33 #x 44.5
m11copy = m1[0].copy()
m22copy = m2[0].copy()


In [106]:
def layer_creation(t, us, m11copy):
    value = int(n/surface_m1)
    times = value*value - 1

    x = 0.0
    y = 0.0

    c = 1

    for i in range(0, times):
        m1copy = m11copy.copy()
        m1copy.center

        if c == value:
            y = y + surface_m1
            x = 0.0
            m1copy.moveBy([[ x,    y,  0.0]])
            m1[0].append(m1copy)
            c = 0


        else: #In this step I set the squares in first column
            x += surface_m1
            m1copy.moveBy([[ x,    y,  0.0]])
            m1[0].append(m1copy)

        c += 1
        
    return(m1[0])

In [107]:
uplayer = layer_creation(30, 100, m11copy)
lowlayer = layer_creation(30, 100, m22copy)

In [108]:
uplayer.center
lowlayer.center

lowlayer.moveBy([[ 0.0,    0.0,   20]])
uplayer.append(lowlayer)

array([None], dtype=object)

In [109]:
uplayer.view('VMD')